# Stem

This notebook that analysis the stem.

---------------

##### Imports

In [1]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

In [2]:
# Import modules.
import os
import math
import time
import trimesh
import shapely
import pymeshfix
import subprocess
import alphashape
import numpy as np
import open3d as o3d
import logging as log
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.spatial import KDTree
from descartes import PolygonPatch
from shapely.geometry import Polygon
from scipy.spatial import ConvexHull
from plyfile import PlyData, PlyElement
from skimage.measure import CircleModel, ransac
from sklearn.neighbors import NearestNeighbors
import utils.math_utils as math_utils
import utils.plot_utils as plot_utils
import utils.o3d_utils as o3d_utils
from misc.quaternion import Quaternion

In [51]:
class Stem:
    """ This class implements the AdTree delft repository.
    Attributes:
        ---
    """

    def __init__(self):
        pass
    
    def _fit_cylinders(self, pcd, skeleton_points, num_neighbours):
        """
        Fits a 3D line to the skeleton points cluster provided.
        Uses this line as the major axis/axial vector of the cylinder to be fitted.
        Fits a series of circles perpendicular to this axis to the point cloud of this particular stem segment.
        Args:
            pcd: The cluster of points belonging to the segment of the branch.
            skeleton_points: A single cluster of skeleton points which should represent a segment of a tree/branch.
            num_neighbours: The number of skeleton points to use for fitting each circle in the segment. lower numbers
                            have fewer points to fit a circle to, but higher numbers are negatively affected by curved
                            branches. Recommend leaving this as it is.
        Returns:
            cyl_array: a numpy array based representation of the fitted circles/cylinders.
        """

        point_cloud = np.array(pcd.points)
        skeleton_points = skeleton_points[:, :3]
        cyl_array = np.zeros((0, 5))
        line_centre = np.mean(skeleton_points[:, :3], axis=0) # TODO center of bounding box...
        _, _, vh = np.linalg.svd(line_centre - skeleton_points)
        line_v_hat = vh[0] / np.linalg.norm(vh[0])

        if skeleton_points.shape[0] <= num_neighbours:
            group = skeleton_points
            line_centre = np.mean([np.min(group[:, :3], axis=0), np.max(group[:, :3], axis=0)], axis=0)
            length = np.linalg.norm(np.max(group, axis=0) - np.min(group, axis=0))
            plane_slice = point_cloud[
                np.linalg.norm(abs(line_v_hat * (point_cloud - line_centre)), axis=1) < (length / 2)
            ]  # calculate distances to plane at centre of line.
            if plane_slice.shape[0] > 0:
                cylinder = fit_circle_3D(plane_slice, line_v_hat)
                cyl_array = np.vstack((cyl_array, cylinder))
        else:
            for i in tqdm(range(skeleton_points.shape[0])):
                if skeleton_points.shape[0] > num_neighbours:
                    nn = NearestNeighbors()
                    nn.fit(skeleton_points)
                    starting_point = np.atleast_2d(skeleton_points[np.argmin(skeleton_points[:, 2])])
                    group = skeleton_points[nn.kneighbors(starting_point, n_neighbors=num_neighbours)[1][0]]
                    line_centre = np.mean(group[:, :3], axis=0)
                    length = np.linalg.norm(np.max(group, axis=0) - np.min(group, axis=0))
                    plane_slice = point_cloud[
                        np.linalg.norm(abs(line_v_hat * (point_cloud - line_centre)), axis=1) < (length / 2)
                    ]  # calculate distances to plane at centre of line.
                    if plane_slice.shape[0] > 0:
                        cylinder = fit_circle_3D(plane_slice, line_v_hat)
                        cyl_array = np.vstack((cyl_array, cylinder))
                    skeleton_points = np.delete(skeleton_points, np.argmin(skeleton_points[:, 2]), axis=0)
        
        return cyl_array
        
    def _construct_skeleton(self, pcd, slice_thickness=0.15):

        points = np.asarray(pcd.points)
        min_z, max_z = min(points[:, 2])-slice_thickness, max(points[:, 2])+slice_thickness
        bins = np.arange(min_z, max_z, slice_thickness)
        slice_ind = np.digitize(points[:,2], bins, right=True)

        skeleton_points = np.zeros((0, 3))
        for i in np.unique(slice_ind):
            new_slice = points[slice_ind==i]
            if new_slice.shape[0] > 0:
                median = np.median(new_slice, axis=0)
                skeleton_points = np.vstack((skeleton_points, median))

        return skeleton_points

    def _plot_stem_rims(self, cyl_array, resolution=15, cloud=None):
    
        geometries = []
        for result in cyl_array:
            c = result[:3]
            r = result[6]

            line_points = np.zeros((0,3))
            for i in range(resolution):
                phi = i*np.pi/(resolution/2)
                line_points = np.vstack((line_points, (c[0] + r*np.cos(phi), c[1] + r*np.sin(phi), c[2])))

            line_set = o3d.geometry.LineSet()
            line_set.points = o3d.utility.Vector3dVector(line_points)
            line_set.lines = o3d.utility.Vector2iVector([(i,i+1) for i in range(resolution-1)])
            line_set.colors = o3d.utility.Vector3dVector(np.zeros((resolution-1,3)))
            geometries.append(line_set)

        if cloud is not None:
            geometries.append(cloud)

        o3d.visualization.draw_geometries(geometries)

    def _statistics(self, cyl_array):

        stats = {
                "bottom_radius": np.round(cyl_array[0,3],2),
                "top_radius": np.round(np.mean(cyl_array[:,3]),2),
                "mean_radius": np.round(cyl_array[-1,3],2),
                "height": np.round(cyl_array[-1,2]-cyl_array[0,2],2),
                "angle": np.round(math_utils.vector_angle(cyl_array[-1,:3] - cyl_array[0,:3]),2)
        }

        print('Stem statistics:')
        print(f"- height: {stats['height']}")
        print(f"- radius: {stats['bottom_radius']} (bottom) / {stats['top_radius']} (top) / {stats['mean_radius']} (mean)")
        print(f"- angle: {stats['angle']}")

        return stats

    def _stem_to_cyl_mesh(self, cyl_array, resolution=15):
        stem_cyl = trimesh.creation.cylinder(radius=np.mean(cyl_array[:,3]), sections=resolution, segment=cyl_array[[0,-1],:3]).as_open3d
        stem_cyl.paint_uniform_color([0.348, 0.11, 0])
        stem_cyl.compute_vertex_normals()
        return stem_cyl
        
    def _stem_to_mesh(self, cyl_array, resolution=15):

        circle_fits = [(rim[:3], rim[3]) for rim in cyl_array]
        num_slices = len(circle_fits)

        # compute directional axis
        Z = np.array([0, 0, 1], dtype=float)
        centers = np.array([c for c, r in circle_fits])
        axis = centers[-1] - centers[0]
        l = np.linalg.norm(axis)
        if l <= 1e-12:
            axis=Z

        rot = Quaternion.fromData(Z, axis).to_matrix()

        # create vertices
        angles = [2*math.pi*i/float(resolution) for i in range(resolution)]
        rim = np.array([[math.cos(theta), math.sin(theta), 0.0]
            for theta in angles])
        rim = np.dot(rot, rim.T).T

        rims = np.array([
            rim * r + c
            for c, r in circle_fits], dtype=float)
        rims = rims.reshape((-1,3))

        vertices = np.vstack([[centers[0], centers[-1]], rims ])

        # create faces
        bottom_fan = np.array([
            [0, (i+1)%resolution+2, i+2]
            for i in range(resolution) ], dtype=int)

        top_fan = np.array([
            [1, i+2+resolution*(num_slices-1), (i+1)%resolution+2+resolution*(num_slices-1)]
            for i in range(resolution) ], dtype=int)

        slice_fan = np.array([
            [[2+i, (i+1)%resolution+2, i+resolution+2],
                [i+resolution+2, (i+1)%resolution+2, (i+1)%resolution+resolution+2]]
            for i in range(resolution) ], dtype=int)
        slice_fan = slice_fan.reshape((-1, 3), order="C")

        side_fan = np.array([
            slice_fan + resolution*i 
            for i in range(num_slices-1)], dtype=int)
        side_fan = side_fan.reshape((-1, 3), order="C") 

        faces = np.vstack([bottom_fan, top_fan, side_fan])

        # create mesh
        mesh = trimesh.base.Trimesh(vertices, faces).as_open3d

        return mesh

    def model(self, pcd, plot=False):
        "3D model skeleton"
        
        # Construct skeleton 
        skeleton_points = self._construct_skeleton(pcd)

        # Fit cylinders
        cyl_array = self._fit_cylinders(pcd, skeleton_points, 3)

        if plot:
            self._plot_stem_cyl(cyl_array, cloud=pcd)

        stats = self._statistics(cyl_array)

        # simple mesh
        cyl_mesh = self._stem_to_cyl_mesh(cyl_array)
        detailed_mesh = self._stem_to_mesh(cyl_array)

        return [detailed_mesh, cyl_mesh], stats


def fit_circle_2D(points):

    CCI = 0
    r = 0
    xc, yc = np.mean(points[:, :2], axis=0)

    # Fit circle in new 2D coords with RANSAC
    if points.shape[0] >= 20:

        model_robust, inliers = ransac(
            points[:, :2],
            CircleModel,
            min_samples=int(points.shape[0] * 0.3),
            residual_threshold=0.05,
            max_trials=1000,
        )
        xc, yc = model_robust.params[0:2]
        r = model_robust.params[2]
        CCI = math_utils.circumferential_completeness_index([xc, yc], r, points[:, :2])

    if CCI < 0.3:
        r = 0
        xc, yc = np.mean(points[:, :2], axis=0)
        CCI = 0

    return CCI, r, xc, yc


def fit_circle_3D(points, V):
    """Fits a circle using Random Sample Consensus (RANSAC) to a set of points in a plane perpendicular to vector V."""

    P = points[:, :3]
    P_mean = np.mean(P, axis=0)
    P_centered = P - P_mean
    normal = V / np.linalg.norm(V)
    if normal[2] < 0:  # if normal vector is pointing down, flip it around the other way.
        normal = normal * -1

    # Project points to coords X-Y in 2D plane and fit circle
    P_xy = math_utils.rodrigues_rot(P_centered, normal, [0, 0, 1])
    CCI, r, xc, yc = fit_circle_2D(P_xy)

    # Transform circle center back to 3D coords
    cyl_centre = math_utils.rodrigues_rot(np.array([[xc, yc, 0]]), [0, 0, 1], normal) + P_mean
    cyl_output = np.array([[cyl_centre[0, 0], cyl_centre[0, 1], cyl_centre[0, 2], r, CCI]])

    return cyl_output


##### Experimental

In [56]:
# Load point cloud data
source = 'x'

if source == 'ahn':
    pcd = o3d_utils.read_las('../datasets/single_selection/single_121913_487434_AHN.las')
elif source == 'cyclo':
    pcd = o3d_utils.read_las('../datasets/single_selection/single_121913_487434_Cyclo.las')
else:
    pcd = o3d_utils.read_las('../datasets/single_selection/single_121913_487434_Sonarski.las')

Point cloud of 929858 points, (13.5x13.2x15.1), and 0.021 point density.


In [58]:
stem_cloud = pcd.select_by_index(np.where(np.asarray(pcd.points)[:,2]<4.8)[0])
o3d.visualization.draw_geometries([stem_cloud])

In [59]:
stemAnalysis = Stem()

In [55]:
meshes, stats = stemAnalysis.model(stem_cloud)

100%|██████████| 23/23 [00:06<00:00,  3.55it/s]

Stem statistics:
- height: 2.8
- radius: 0.36 (bottom) / 0.29 (top) / 0.27 (mean)
- angle: 5.06
